# Pretrained CNN - Feature Extraction with Data Augmentation
<br>In diesem Code Notebook behandeln wir Feature Extraction mit Data Augmentation, dies sollte nur auf Server mit GPU laufen.

### Load Packages

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

import matplotlib
import matplotlib.pyplot as plt # plotting library
%matplotlib inline
matplotlib.get_backend()

2023-04-18 16:12:26.640628: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 16:12:26.643718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 16:12:26.691477: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 16:12:26.692340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 16:12:27.472728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

'module://matplotlib_inline.backend_inline'

### Load Path Files

In [2]:

original_dataset_dir = '../data/train'

base_dir = '../data/cats_and_dogs_2k'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

### Data Preparation with Data Augmentation

In [3]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                                                                rescale=1./255,
                                                                rotation_range=40,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True
                                                             )

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Load Convolutional Base

In [4]:
conv_base = tf.keras.applications.VGG16(weights='imagenet',
                                        include_top=False,
                                        input_shape=(150,150,3))
print(conv_base.summary())

2023-04-18 16:12:29.115097: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

### Model building

In [5]:
model = tf.keras.models.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Compile model

In [6]:
model.compile(loss='binary_crossentropy',
              metrics=['acc'],
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5))

### Fit Model

In [7]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data= validation_generator,
    validation_steps=50
)

Epoch 1/30


/tmp/ipykernel_1691883/3385942099.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2023-04-18 16:12:30.077690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - ETA: 0s - loss: 0.6073 - acc: 0.6580

2023-04-18 16:13:30.940128: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 74s 720ms/step - loss: 0.6073 - acc: 0.6580 - val_loss: 0.3206 - val_acc: 0.8910
Epoch 2/30
100/100 [==============================] - 71s 707ms/step - loss: 0.3704 - acc: 0.8395 - val_loss: 0.2661 - val_acc: 0.8960
Epoch 3/30
100/100 [==============================] - 71s 711ms/step - loss: 0.2707 - acc: 0.8920 - val_loss: 0.1663 - val_acc: 0.9320
Epoch 4/30
100/100 [==============================] - 71s 707ms/step - loss: 0.2353 - acc: 0.9040 - val_loss: 0.1675 - val_acc: 0.9360
Epoch 5/30
100/100 [==============================] - 71s 712ms/step - loss: 0.1908 - acc: 0.9190 - val_loss: 0.1144 - val_acc: 0.9520
Epoch 6/30
100/100 [==============================] - 71s 708ms/step - loss: 0.1796 - acc: 0.9305 - val_loss: 0.3204 - val_acc: 0.9080
Epoch 7/30
100/100 [==============================] - 71s 709ms/step - loss: 0.1391 - acc: 0.9395 - val_loss: 0.1080 - val_acc: 0.9630
Epoch 8/30
100/100 [==============================] - 71s 708ms/st

### Fine_Tune

In [8]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

### Compile model

In [9]:
model.compile(loss='binary_crossentropy',
              metrics=['acc'],
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5))

### Fit Model

In [10]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data= validation_generator,
    validation_steps=50
)

Epoch 1/30


/tmp/ipykernel_1691883/3385942099.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2023-04-18 16:48:42.860294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - ETA: 0s - loss: 0.0223 - acc: 0.9925

2023-04-18 16:49:14.775488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 45s 442ms/step - loss: 0.0223 - acc: 0.9925 - val_loss: 0.1355 - val_acc: 0.9710
Epoch 2/30
100/100 [==============================] - 44s 439ms/step - loss: 0.0170 - acc: 0.9955 - val_loss: 0.1118 - val_acc: 0.9750
Epoch 3/30
100/100 [==============================] - 44s 442ms/step - loss: 0.0132 - acc: 0.9950 - val_loss: 0.1031 - val_acc: 0.9750
Epoch 4/30
100/100 [==============================] - 44s 442ms/step - loss: 0.0149 - acc: 0.9945 - val_loss: 0.1060 - val_acc: 0.9770
Epoch 5/30
100/100 [==============================] - 44s 441ms/step - loss: 0.0234 - acc: 0.9930 - val_loss: 0.1359 - val_acc: 0.9710
Epoch 6/30
100/100 [==============================] - 44s 442ms/step - loss: 0.0139 - acc: 0.9945 - val_loss: 0.1315 - val_acc: 0.9740
Epoch 7/30
100/100 [==============================] - 44s 441ms/step - loss: 0.0133 - acc: 0.9960 - val_loss: 0.1494 - val_acc: 0.9660
Epoch 8/30
100/100 [==============================] - 44s 439ms/st

### Save model

In [11]:
model.save('cats_and_dogs_pretrained_with_fine_tuning.h5')